In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Install YOLOv8

In [ ]:
# YOLOv8와 YOLOv8 실행에 필요한 라이브러리 설치
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.4/660.4 kB 9.3 MB/s eta 0:00:00


In [ ]:
import ultralytics

ultralytics.checks()

Ultralytics YOLOv8.0.226 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 26.9/166.8 GB disk)


# 2. Downloading the Dataset

In [ ]:
# roboflow에서 전처리된 데이터셋 코드로 가져오기
# OR
# 내드라이브에 custom dataset 올려놓기(구글 마운트하면서 가져와질 것임)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.226, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to dataset-3 in yolov8:: 100%|██████████| 6424/6424 [00:00<00:00, 8851.83it/s]


# 3. Load a pre-trained model

In [ ]:
from ultralytics import YOLO

# YOLO 모델 초기화
# model = YOLO('yolov8n.pt')
model = YOLO('yolov8s.pt')
# model = YOLO('yolov8m.pt')
# model = YOLO('yolov8l.pt')
# model = YOLO('yolov8x.pt')

100%|██████████| 21.5M/21.5M [00:00<00:00, 59.5MB/s]


#### MS COCO Dataset에 정의되어 있는 클래스 개수와 종류 확인

In [ ]:
print(type(model.names), len(model.names))

print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

# 4. Training

## 1) data.yaml 파일 수정

In [ ]:
# 불러온 파일 중 yaml 파일을 확인
!cat /content/bebeface-1/data.yaml

names:
- back
- negative
- neutral
- positive
nc: 4
roboflow:
  license: CC BY 4.0
  project: bebeface-fhspj
  url: https://universe.roboflow.com/kosakorea-softwareindustry-association/bebeface-fhspj/dataset/3
  version: 3
  workspace: kosakorea-softwareindustry-association
test: ../test/images
train: dataset-3/train/images
val: dataset-3/valid/images


In [ ]:
# PyYAML 설치
!pip install PyYAML

In [ ]:
# 필요한 module import
import yaml

In [ ]:
augment = {
    'rotate': 0.1,
    'brightness': 0.1,
    'saturation': 0.1,
    'contrast' : 0.1,
    'blur': 0.1,
    'fliplr': 0.1
}


data = {'train' : '/content/bebeface-1/train/images',
        'val' : '/content/bebeface-1/valid/images',
        'test' : '/content/bebeface-1/test/images',
        'names' : ['back','negative','neutral', 'positive'],
        'nc' : 4,
        'augment' : augment}

with open('/content/bebeface-1/data.yaml','w') as f:
  yaml.dump(data,f)

with open('/content/bebeface-1/data.yaml', 'r') as f:
  bebeFace_yaml = yaml.safe_load(f)
  display(bebeFace_yaml)

{'augment': {'blur': 0.1,
  'brightness': 0.1,
  'contrast': 0.1,
  'fliplr': 0.1,
  'rotate': 0.1,
  'saturation': 0.1},
 'names': ['back', 'negative', 'neutral', 'positive'],
 'nc': 4,
 'test': '/content/dataset-3/test/images',
 'train': '/content/dataset-3/train/images',
 'val': '/content/dataset-3/valid/images'}

## 2) 불러온 pretraind-model인 yolo 모델을 기반으로, 학습 진행

In [ ]:
# yolov8 모델을 기반으로, 우리가 만든 data.yaml으로 학습 진행
model.train(data = '/content/bebeface-1/data.yaml',
            epochs=100,
            patience=20,
            batch=32,
            imgsz=540,
            project='/content/drive/MyDrive/KOSA3/project/result',
            name='sample',
            exist_ok=True)

Ultralytics YOLOv8.0.226 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/dataset-3/data.yaml, epochs=100, patience=20, batch=32, imgsz=540, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/KOSA3/project/result, name=sample, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

100%|██████████| 755k/755k [00:00<00:00, 4.07MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 22.5MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[540] must be multiple of max stride 32, updating to [544]


...

                   all        598        602       0.77      0.785      0.786      0.622
Stopping training early as no improvement observed in last 20 epochs. Best results observed at epoch 39, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



59 epochs completed in 0.489 hours.
Optimizer stripped from /content/drive/MyDrive/KOSA3/project/result/sample/weights/last.pt, 22.7MB
Optimizer stripped from /content/drive/MyDrive/KOSA3/project/result/sample/weights/best.pt, 22.6MB

Validating /content/drive/MyDrive/KOSA3/project/result/sample/weights/best.pt...
Ultralytics YOLOv8.0.226 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:05<00:00,  1.67it/s]


                   all        598        602      0.798      0.772      0.895      0.721
                  back        598         76      0.992      0.934      0.969      0.831
              negative        598         79      0.409      0.911      0.892      0.722
               neutral        598        157      0.853      0.822       0.89      0.675
              positive        598        290      0.941      0.421      0.828      0.657
Speed: 0.1ms preprocess, 3.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/drive/MyDrive/KOSA3/project/result/sample


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d02a3a144f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [ ]:
# 학습된 결과물 출력
print(type(model.names),len(model.names))
print(model.names)

<class 'dict'> 4
{0: 'back', 1: 'negative', 2: 'neutral', 3: 'positive'}


#### pretrained-model 따로 다운 받을 필요 없이 바로 학습시키고 싶을 때 사용하는 코드

In [ ]:
# 밑 코드는 model이 항상 yolov8n.pt로 되어있음
!yolo task=detect mode=train model=yolov8m.pt data='/content/bebeface-6/data.yaml' epochs=1 patience=12 batch=32 imgsz=640 project='/content/drive/MyDrive/KOSA3/project/result' name=sample exist_ok=True

Ultralytics YOLOv8.0.221 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/bebeface-6/data.yaml, epochs=1, patience=12, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/KOSA3/project/result, name=sample, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

# 4. Validate



> mAP(Mean Average Precision) : 각 클래스에 대한 Average Precision을 평균내어 얻은 값으로, 모든 클래스에 대한 성능을 종합적으로 나타냄
- 높은 mAP는 모델이 각 클래스에 대해 높은 정밀도와 재현율을 갖고 있다는 것


> Box(Precision, Recall) : 각 클래스에 대한 박스 수준의 정밀도와 재현율
- 클래스 당 높은 박스 수준의 정밀도와 재현율은 모델이 해당 클래스를 얼마나 잘 감지하는지


> mAP50, mAP50-95 : IoU (Intersection over Union) 임계값을 다르게 한 경우의 mAP 값
- 높은 mAP50와 mAP50-95는 모델이 다양한 IoU 임계값에서 좋은 성능을 보이는 것을 나타냅니다.


> Precision, Recall : 모델의 정밀도와 재현율
- 높은 정밀도는 모델이 예측한 결과가 실제로 정답인 비율이 높다는 것
- 높은 재현율은 모든 실제 정답 중에서 모델이 얼마나 많은 것을 찾아냈는지를 나타냅니다.









In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# training 과정에서 평가 지표를 보고 싶으면, 스크롤을 올려서 표를 찾으면 되고,
# 따로 바로 보고싶으면 이 코드를 실행시키면 된다
!yolo task=detect mode=val model=/content/drive/MyDrive/KOSA3/project/result/sample/weights/best.pt data=/content/bebeface-1/data.yaml

Ultralytics YOLOv8.0.226 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/dataset-3/valid/labels.cache... 598 images, 0 backgrounds, 0 corrupt: 100% 598/598 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 38/38 [00:07<00:00,  5.05it/s]
                   all        598        602        0.8      0.772      0.895      0.721
                  back        598         76      0.992      0.934      0.969      0.828
              negative        598         79      0.411      0.911      0.892      0.726
               neutral        598        157      0.854      0.822       0.89      0.675
              positive        598        290      0.942      0.421      0.828      0.656
Speed: 0.4ms preprocess, 6.4ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https:

# 5. Prediction

## 1)방법





> ### 방법1 : CLI



In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/KOSA3/project/result/sample/weights/best.pt conf=0.25 source=/content/drive/MyDrive/KOSA3/project/data/prediction save=True

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25842076 parameters, 0 gradients, 78.7 GFLOPs

image 1/11 /content/drive/MyDrive/KOSA3/project/data/prediction/back1.jpeg: 544x544 1 back, 32.9ms
image 2/11 /content/drive/MyDrive/KOSA3/project/data/prediction/back2.jpeg: 544x544 1 back, 33.0ms
image 3/11 /content/drive/MyDrive/KOSA3/project/data/prediction/neg1.jpeg: 544x544 1 negative, 33.0ms
image 4/11 /content/drive/MyDrive/KOSA3/project/data/prediction/neg2.jpeg: 544x544 1 negative, 32.9ms
image 5/11 /content/drive/MyDrive/KOSA3/project/data/prediction/pos1.jpeg: 544x544 1 positive, 32.9ms
image 6/11 /content/drive/MyDrive/KOSA3/project/data/prediction/pos2.jpeg: 544x544 (no detections), 33.0ms
video 7/11 (1/1416) /content/drive/MyDrive/KOSA3/project/data/prediction/baby1.mov: 544x352 1 neutral, 65.8ms
video 7/11 (2/1416) /content/drive/MyDrive/KOSA3/project/data/prediction/baby1.mov: 544x352 1 neutral, 17.4ms



> ### 방법2



In [ ]:
model = YOLO("/content/drive/MyDrive/KOSA3/project/result/sample/weights/best.pt")

In [ ]:
# source = '내가 테스트할 사진이나 영상 미리 넣어놓기'
results = model.predict(source = "/content/drive/MyDrive/KOSA3/project/data/custom_test_img", save=True)


image 1/6 /content/drive/MyDrive/KOSA3/project/data/custom_test_img/back1.jpeg: 544x544 1 back, 34.0ms
image 2/6 /content/drive/MyDrive/KOSA3/project/data/custom_test_img/back2.jpeg: 544x544 1 back, 33.1ms
image 3/6 /content/drive/MyDrive/KOSA3/project/data/custom_test_img/neg1.jpeg: 544x544 1 negative, 33.0ms
image 4/6 /content/drive/MyDrive/KOSA3/project/data/custom_test_img/neg2.jpeg: 544x544 1 negative, 33.1ms
image 5/6 /content/drive/MyDrive/KOSA3/project/data/custom_test_img/pos1.jpeg: 544x544 1 positive, 33.0ms
image 6/6 /content/drive/MyDrive/KOSA3/project/data/custom_test_img/pos2.jpeg: 544x544 (no detections), 22.0ms
Speed: 2.2ms preprocess, 31.4ms inference, 1.5ms postprocess per image at shape (1, 3, 544, 544)
Results saved to runs/detect/predict2


## 2) 저장

In [ ]:
import shutil

# 복사할 폴더의 경로
source_folder = "/content/runs/detect/predict3"

# Google Drive에 저장할 경로
destination_folder = "/content/drive/MyDrive/KOSA3/project/prediction/predict1" # 이미존재하는 폴더일 때 여기 뒤 숫자 바꾸기

# shutil을 사용하여 폴더 복사
shutil.copytree(source_folder, destination_folder)

print(f"{source_folder} 폴더를 {destination_folder}로 복사 완료")

/content/runs/detect/predict3 폴더를 /content/drive/MyDrive/KOSA3/project/prediction/predict2로 복사 완료


# 6. Export

In [ ]:
model = YOLO("/content/drive/MyDrive/KOSA3/project/result/sample/weights/best.pt")

In [ ]:
model.export(format="onnx", imgsz=[480,640])

Ultralytics YOLOv8.0.223 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.30GHz)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/KOSA3/project/result/sample/weights/best.pt' with input shape (1, 3, 480, 640) BCHW and output shape(s) (1, 8, 6300) (21.6 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 1.2s, saved as '/content/drive/MyDrive/KOSA3/project/result/sample/weights/best.onnx' (42.6 MB)

Export complete (2.8s)
Results saved to /content/drive/MyDrive/KOSA3/project/result/sample/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/KOSA3/project/result/sample/weights/best.onnx imgsz=480,640  
Validate:        yolo val task=detect model=/content/drive/MyDrive/KOSA3/project/result/sample/weights/best.onnx imgsz=480,640 data=/content/bebeFace-3/data.yaml  WARNING ⚠️ non-PyTorch val requires square images, 'imgsz=[480, 640]' will not work. Use exp

'/content/drive/MyDrive/KOSA3/project/result/sample/weights/best.onnx'